In [1]:
import sys
print(sys.prefix)

C:\Users\Schalk\anaconda3


In [7]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import os, getpass

In [10]:
from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec

In [5]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]

In [12]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader

loader = PyPDFLoader(r"C:\Users\Schalk\OneDrive - Columbia Business School\2024\Recruiting\Schalk Burger Resume.pdf")

In [8]:
data = loader.load()

In [9]:
for doc in data:
    if not hasattr(doc, 'metadata'):
        doc.metadata = {}
    doc.metadata["Document Type"] = "Academics"

In [15]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')
print(f'Metadata: {data[0].metadata}')

You have 1 document(s) in your data
There are 3709 characters in your sample document
Here is a sample: SCHALK BURGER  
929 325 9282 , sburger25@gsb.columbia.edu  
 
EDUCATION  
 
COLUMBIA BUSINESS SCHOOL  New York, NY  
MBA , Applied Mathematical Statistics & Data Science  Spring 2025  
Leadership:  Cl
Metadata: {'source': 'C:\\Users\\Schalk\\OneDrive - Columbia Business School\\2024\\Recruiting\\Schalk Burger Resume.pdf', 'page': 0, 'Document Type': 'Academics'}


In [16]:
# We'll split our data into chunks around 500 characters each with a 50 character overlap. These are relatively small.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
docs = text_splitter.split_documents(data)

In [17]:
# Let's see how many small chunks we have
print (f'Now you have {len(docs)} documents')

Now you have 12 documents


In [18]:
model_name = 'text-embedding-ada-002'  
embeddings = OpenAIEmbeddings(  
    model=model_name,  
    openai_api_key=OPENAI_API_KEY
)

vectorstore_from_docs = PineconeVectorStore.from_documents(
        docs,
        index_name=PINECONE_INDEX_NAME,
        embedding=embeddings,
        namespace='schalk-burger'
    )

#docsearch = Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=PINECONE_INDEX_NAME, namespace = 'schalk-burger')

In [14]:
pc.create_index(  
    'quinn',  
    dimension=1536,  # dimensionality of text-embedding-ada-002  
    metric='cosine',  
    spec=ServerlessSpec(cloud='aws', region='us-east-1')  
)  

In [13]:
pc.delete_index('quinn')